In [3]:
import pandas as pd
import networkx as nx
from datetime import datetime
from collections import defaultdict


Suspected Hotspots (Warehouse pairs):
W34 (Andreton) - W70 (Port Meredithmouth): Weight 77
W32 (South Samuel) - W99 (Duncantown): Weight 73
W38 (Josephtown) - W76 (Lake Royport): Weight 71
W38 (Josephtown) - W67 (Johnsonfort): Weight 70
W63 (Brandonburgh) - W68 (Aguilarchester): Weight 70
W19 (Port Jessica) - W98 (Port Seanhaven): Weight 69
W16 (Elizabethfurt) - W36 (Lake Sandrabury): Weight 67
W6 (Salinashaven) - W41 (New Elizabeth): Weight 64
W60 (Matthewtown) - W70 (Port Meredithmouth): Weight 64
W12 (Barbaraport) - W63 (Brandonburgh): Weight 63
W21 (North Heatherfort) - W67 (Johnsonfort): Weight 63
W39 (Joshuaside) - W63 (Brandonburgh): Weight 63
W63 (Brandonburgh) - W70 (Port Meredithmouth): Weight 63
W2 (Rodriguezland) - W63 (Brandonburgh): Weight 62
W9 (North Carrie) - W63 (Brandonburgh): Weight 62
W19 (Port Jessica) - W42 (New Jessica): Weight 62
W19 (Port Jessica) - W76 (Lake Royport): Weight 62
W26 (Staceymouth) - W38 (Josephtown): Weight 62
W26 (Staceymouth) - W77 (Curtisbur

In [5]:

# Load the CSV files
shipments_df = pd.read_csv('shipment_database1.csv')
status_df = pd.read_csv('shipment_status_database1.csv')
locations_df = pd.read_csv('location_database1.csv')

# Initialize data structures
G = nx.Graph()
product_counterfeit_scores = {}
seller_counterfeit_scores = {}
warehouse_connections = defaultdict(set)
warehouse_locations = {}

# 1. Initialize Graph and Warehouse Locations
for _, row in locations_df.iterrows():
    G.add_node(row['Warehouse_ID'])
    warehouse_locations[row['Warehouse_ID']] = row['Warehouse_Location']


In [6]:

# Helper function to parse timestamps
def parse_timestamp(ts):
    return datetime.strptime(ts, '%d-%m-%Y %H:%M')

# 2. Update Counterfeit Scores
def update_counterfeit_scores(shipment_id, is_counterfeit):
    shipment = shipments_df[shipments_df['Shipment_ID'] == shipment_id].iloc[0]
    product_id = shipment['Product_ID']
    seller_id = shipment['Seller_ID']

    if is_counterfeit:
        product_counterfeit_scores[product_id] = product_counterfeit_scores.get(product_id, 0) + 1
        seller_counterfeit_scores[seller_id] = seller_counterfeit_scores.get(seller_id, 0) + 1

# 3. Update Graph Weights
def update_graph_weights(origin, destination, num_counterfeits):
    if origin != destination:
        if not G.has_edge(origin, destination):
            G.add_edge(origin, destination, weight=0)
        G[origin][destination]['weight'] += num_counterfeits

# 4. Detect Hotspots
def detect_hotspots(threshold):
    hotspots = []
    for u, v, data in G.edges(data=True):
        if data.get('weight', 0) > threshold:
            hotspots.append((u, v, data['weight']))
    return hotspots

# 5. Main Algorithm
def main_algorithm():
    # Process shipments
    for _, shipment in shipments_df.iterrows():
        shipment_id = shipment['Shipment_ID']
        status = status_df[status_df['Shipment_ID'] == shipment_id].iloc[0]

        is_counterfeit = status['Counterfeit_Score'] > 0.7  # Adjust threshold as needed
        update_counterfeit_scores(shipment_id, is_counterfeit)

        origin = shipment['Origin']
        destination = shipment['Destination']
        warehouse = shipment['Warehouse_ID']

        # Update warehouse connections
        warehouse_connections[origin].add(warehouse)
        warehouse_connections[destination].add(warehouse)

        num_counterfeits = int(status['Counterfeit_Score'] * 10)  # Convert score to a number of counterfeits

        # Update graph weights for connected warehouses
        for w1 in warehouse_connections[origin]:
            for w2 in warehouse_connections[destination]:
                update_graph_weights(w1, w2, num_counterfeits)

    # Detect hotspots
    hotspot_threshold = 50  # Adjust as needed
    hotspots = detect_hotspots(hotspot_threshold)

    return hotspots, product_counterfeit_scores, seller_counterfeit_scores



In [7]:

# Run the algorithm
hotspots, product_scores, seller_scores = main_algorithm()



In [18]:
def compare_paths(source, des):
    print(nx.astar_path(G, source, des,weight="weight"))
    cost = nx.astar_path_length(G, source, des, weight="weight")
    print(cost)

['W1', 'W40', 'W43', 'W19']
1


In [ ]:

# Print results
print("Suspected Hotspots (Warehouse pairs):")
for hotspot in sorted(hotspots, key=lambda x: x[2], reverse=True):
    w1, w2, weight = hotspot
    print(f"{w1} ({warehouse_locations.get(w1, 'Unknown')}) - {w2} ({warehouse_locations.get(w2, 'Unknown')}): Weight {weight}")

print("\nTop 5 Products with Highest Counterfeit Scores:")
for product, score in sorted(product_scores.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"{product}: {score}")

print("\nTop 5 Sellers with Highest Counterfeit Scores:")
for seller, score in sorted(seller_scores.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"{seller}: {score}")

# Save hotspots to a CSV file
hotspots_df = pd.DataFrame(hotspots, columns=['H1', 'H2', 'Weight'])
hotspots_df.to_csv('hotspots.csv', index=False)

print("\nHotspots have been saved to 'hotspots.csv'")